In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [2]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [3]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (파일 이름으로 자동 탐색)
def find_file(filename, search_path='.'):
    # 주어진 파일명을 search_path부터 재귀적으로 탐색하여 경로 반환
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# 상위 디렉토리부터 탐색 시작
# 현재 노트북의 상위 디렉토리부터 탐색 시작
# base_path = pathlib.Path.cwd().parent.parent
# 현재 디렉토리부터 탐색 시작
base_path = pathlib.Path.cwd()

update_profile_logic_schema_path = find_file("Update_Profile_Logic_Object.json", base_path)
update_profile_logic_system_prompt_path = find_file("Update_Profile_Logic_SystemPrompt.md", base_path)

if not update_profile_logic_schema_path or not update_profile_logic_system_prompt_path:
    raise FileNotFoundError("Update Profile Logic 프롬포트 파일을 찾을 수 없습니다")

with open(update_profile_logic_schema_path, 'r', encoding='utf-8') as f:
    update_profile_logic_schema = json.load(f)

with open(update_profile_logic_system_prompt_path, 'r', encoding='utf-8') as f:
    update_profile_logic_system_prompt = f.read()

In [4]:
# contents 생성

# 가짜 입력값
current_profile = {
    "learning_goal": {
        "focus_areas": ["Policy Gradient", "Clipping method"],
        "target_depth": "Deep Understanding",
        "question_modality": "Mathematical"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": None,
        "scenario_based": None
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}

user_answer = "한국어를 난 더 선호하고, 예시를 들어서 설명해주면 더 좋아요."

# contents 생성시, 딕셔너리는 문자열로 바꾼다
current_session_profile_str = json.dumps(current_profile, ensure_ascii=False)

profile_Update_agent_contents = [
    "Current Profile: " + current_session_profile_str,
    "User Answer: " + user_answer
]

In [5]:
# Response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = profile_Update_agent_contents,
    config={
        "system_instruction": update_profile_logic_system_prompt,
        "response_mime_type": "application/json",
        "response_schema": update_profile_logic_schema
    })

In [6]:
print(response.text)

{
  "learning_goal": {
    "focus_areas": [
      "Policy Gradient",
      "Clipping method"
    ],
    "target_depth": "Deep Understanding",
    "question_modality": "Mathematical"
  },
  "user_status": {
    "proficiency_level": "Intermediate",
    "weakness_focus": true
  },
  "interaction_style": {
    "language_preference": "Korean_with_Korean_Terms",
    "scenario_based": true
  },
  "feedback_preference": {
    "strictness": "Strict",
    "explanation_depth": "Detailed_with_Examples"
  },
  "scope_boundary": "Lecture_Material_Only"
}
